<a href="https://colab.research.google.com/github/adeshmukh/gaiip/blob/main/notebooks/Intro_to_Agentic_Architectures_VoyagerAGI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voyager AGI

This notebook contains an illustrative implementation of the Voyager AGI architecture.



## Setup

In [ ]:
%%capture

!pip install langchain
!pip install langchain-community
!pip install langgraph
!pip install langchainhub

!pip install langchain_openai
!pip install langchain-anthropic

!pip install openai
!pip install tavily-python
!pip install google-search-results
!pip install wikipedia
!pip install duckduckgo-search


In [ ]:
from google.colab import userdata
import os


key_names = [
    'ANTHROPIC_API_KEY',
    'OPENAI_API_KEY',
    'PINECONE_API_KEY',
    'SERPAPI_API_KEY',
    'TAVILY_API_KEY',
  ]

for key_name in key_names:
    os.environ[key_name] = userdata.get(key_name)

In [ ]:
import logging

logger = logging.getLogger('colab')

logging.basicConfig()

### Model selection

Uncomment the appropriate line to select the model. This notebook should work with any of these models.

In [ ]:
# OpenAI models
openai_model_name = 'gpt-3.5-turbo'  # Inexpensive, fast, good for simple tasks
# openai_model_name = 'gpt-4o'         # Medium cost, fastest

# Anthropic Claude3 models
anthropic_model_name = 'claude-3-sonnet-20240229'  # Medium cost, Best performance on benchmarks
# anthropic_model_name = 'claude-3-opus-20240229'  # Most expensive
# anthropic_model_name = 'claude-3-haiku-20240307' # Leat expensive


-----------------------------------------

### Implemenation with simulated LLM

First, we implement the core Voyager logic and plug in a class to simulate LLM interactions.

In [ ]:
import random

class VoyagerAgent:
    def __init__(self, llm, environment):
        self.llm = llm
        self.environment = environment
        self.inventory = []
        self.skills = set()

    def generate_plan(self):
        prompt = f"""
        Current inventory: {self.inventory}.
        Current skills: {self.skills}.
        Generate a plan to craft a new item.
        """

        return self.llm.generate_plan(prompt)

    def execute_plan(self, plan):
        code = self.llm.generate_code(plan)
        local_vars = {
            'agent': self,
            'environment': self.environment
        }
        exec(code, globals(), local_vars)
        new_inventory = local_vars.get('new_inventory', [])
        new_skills = local_vars.get('new_skills', set())
        self.update_state(new_inventory, new_skills)

    def update_state(self, new_items, new_skills):
        self.inventory.extend(new_items)
        self.skills.update(new_skills)

    def reflect(self, outcome):
        reflection = self.llm.generate_reflection(f"Reflect on the outcome: {outcome}")
        print(f"Agent reflection: {reflection}")


class MinecraftEnvironment:
    def __init__(self, wood=100, stone=10, iron=50):
        self.world = {"wood": wood, "stone": stone, "iron": iron}

    def get_item(self, item):
      """Returns True if the item is available and consumed, False otherwise."""

      if item in self.world and self.world[item] > 0:
          self.world[item] -= 1
          return True

      return False

    def craft_item(self, recipe):
        """Simplified crafting logic"""

        # Check if all ingredients required for the recipe are available.
        return all(self.get_item(ingredient) for ingredient in recipe)

# Simulated LLM for demonstration purposes
class SimpleLLM:
    def generate_plan(self, prompt):
        return "1. Gather wood\n2. Craft a wooden pickaxe\n3. Mine stone\n4. Craft a stone pickaxe"

    def generate_code(self, plan):
        return """
def execute_plan(agent, environment):
    new_inventory = []
    new_skills = set()

    if environment.get_item("wood"):
        new_inventory.append("wood")
    if "wood" in agent.inventory or "wood" in new_inventory:
        if environment.craft_item(["wood", "wood"]):
            new_inventory.append("wooden_pickaxe")
            new_skills.add("woodworking")
    if "wooden_pickaxe" in agent.inventory or "wooden_pickaxe" in new_inventory:
        if environment.get_item("stone"):
            new_inventory.append("stone")
    if ("stone" in agent.inventory or "stone" in new_inventory) and ("woodworking" in agent.skills or "woodworking" in new_skills):
        if environment.craft_item(["stone", "wood"]):
            new_inventory.append("stone_pickaxe")
            new_skills.add("stoneworking")

    return new_inventory, new_skills

new_inventory, new_skills = execute_plan(agent, environment)
"""

    def generate_reflection(self, prompt):
        return "The plan was executed successfully, resulting in new items crafted and skills learned."


Put together the pieces

In [ ]:
llm = SimpleLLM()
env = MinecraftEnvironment(wood=20, stone=0, iron=5)
agent = VoyagerAgent(llm, env)

def step_forward():
  plan = agent.generate_plan()
  print(f"--- Generated plan:\n{plan}")

  agent.execute_plan(plan)

  print(f"--- Updated inventory: {agent.inventory}")
  print(f"--- Updated skills: {agent.skills}")
  print("==========================")
  agent.reflect("New items crafted and skills learned")

In [ ]:
# In reality, this could be an infinite loop for continuous exploration
for i in range(2):
  step_forward()

--- Generated plan:
1. Gather wood
2. Craft a wooden pickaxe
3. Mine stone
4. Craft a stone pickaxe
--- Updated inventory: ['wood', 'wooden_pickaxe']
--- Updated skills: {'woodworking'}
Agent reflection: The plan was executed successfully, resulting in new items crafted and skills learned.
--- Generated plan:
1. Gather wood
2. Craft a wooden pickaxe
3. Mine stone
4. Craft a stone pickaxe
--- Updated inventory: ['wood', 'wooden_pickaxe', 'wood', 'wooden_pickaxe']
--- Updated skills: {'woodworking'}
Agent reflection: The plan was executed successfully, resulting in new items crafted and skills learned.


### Implementation with a real LLM

Now we will attempt to use an actual LLM instead of the simulated one in the prior example.

Also, we will use LangChain as the framework for this implementation.

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict


# Define the MinecraftEnvironment
class MinecraftEnvironment:
    def __init__(self):
        self.resources = {
            "wood": 0,
            "stone": 0,
            "iron": 0,
            "diamond": 0,
            "coal": 0
        }
        self.crafted_items = {
            "wooden_pickaxe": 0,
            "stone_pickaxe": 0,
            "iron_pickaxe": 0,
            "furnace": 0
        }

    def gather_resource(self, resource: str, amount: int):
        if resource in self.resources:
            self.resources[resource] += amount
            return f"Gathered {amount} {resource}"
        return f"Cannot gather {resource}"

    def craft_item(self, item: str):
        if item == "wooden_pickaxe" and self.resources["wood"] >= 3:
            self.resources["wood"] -= 3
            self.crafted_items["wooden_pickaxe"] += 1
            return "Crafted wooden pickaxe"
        elif item == "stone_pickaxe" and self.resources["wood"] >= 2 and self.resources["stone"] >= 3:
            self.resources["wood"] -= 2
            self.resources["stone"] -= 3
            self.crafted_items["stone_pickaxe"] += 1
            return "Crafted stone pickaxe"
        elif item == "iron_pickaxe" and self.resources["wood"] >= 2 and self.resources["iron"] >= 3:
            self.resources["wood"] -= 2
            self.resources["iron"] -= 3
            self.crafted_items["iron_pickaxe"] += 1
            return "Crafted iron pickaxe"
        elif item == "furnace" and self.resources["stone"] >= 8:
            self.resources["stone"] -= 8
            self.crafted_items["furnace"] += 1
            return "Crafted furnace"
        return f"Cannot craft {item}"

    def get_state(self):
        return f"Resources: {self.resources}\nCrafted items: {self.crafted_items}"

# Define the Plan model
class Action(BaseModel):
    action_type: str = Field(description="Type of action: 'gather' or 'craft'")
    target: str = Field(description="Resource to gather or item to craft")
    amount: int = Field(description="Amount to gather (for 'gather' action)")

class Plan(BaseModel):
    actions: List[Action] = Field(description="List of actions to perform")

# Create the LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Create the plan generation prompt
plan_prompt = ChatPromptTemplate.from_template(
    """You are an AI agent in a simplified Minecraft environment.
    Your goal is to create a plan to gather resources and craft items.

    Current state:
    {current_state}

    Goal: {goal}

    Create a plan to achieve the goal. The plan should be a list of actions.
    Each action should be either 'gather' (to collect resources) or 'craft' (to create items).

    Output the plan as a JSON object with the following structure:
    {format_instructions}
    """
)

# Create the plan generation chain
plan_parser = PydanticOutputParser(pydantic_object=Plan)
plan_chain = LLMChain(
    llm=llm,
    prompt=plan_prompt,
    output_parser=plan_parser
)

# Function to execute the plan
def execute_plan(env: MinecraftEnvironment, plan: Plan):
    results = []
    for action in plan.actions:
        if action.action_type == "gather":
            result = env.gather_resource(action.target, action.amount)
        elif action.action_type == "craft":
            result = env.craft_item(action.target)
        else:
            result = f"Unknown action: {action.action_type}"
        results.append(result)
    return results

# Main loop
def main():
    env = MinecraftEnvironment()
    print("Starting Voyager AGI simulation...")

    goals = [
        "Craft a stone pickaxe",
        "Gather 10 iron ore",
        "Craft an iron pickaxe",
        "Gather 5 diamonds"
    ]

    for goal in goals:
        print(f"\nCurrent goal: {goal}")
        print("Current state:")
        print(env.get_state())

        # Generate plan
        plan = plan_chain.run(
            current_state=env.get_state(),
            goal=goal,
            format_instructions=plan_parser.get_format_instructions()
        )

        print("\nGenerated plan:")
        for action in plan.actions:
            print(f"- {action.action_type} {action.target}" + (f" (amount: {action.amount})" if action.action_type == "gather" else ""))

        # Execute plan
        print("\nExecuting plan:")
        results = execute_plan(env, plan)
        for result in results:
            print(f"- {result}")

        # Reflect on outcome
        print("\nReflection:")
        print(env.get_state())

        if goal == "Craft a stone pickaxe" and env.crafted_items["stone_pickaxe"] > 0:
            print("Goal achieved: Crafted a stone pickaxe")
        elif goal == "Gather 10 iron ore" and env.resources["iron"] >= 10:
            print("Goal achieved: Gathered 10 iron ore")
        elif goal == "Craft an iron pickaxe" and env.crafted_items["iron_pickaxe"] > 0:
            print("Goal achieved: Crafted an iron pickaxe")
        elif goal == "Gather 5 diamonds" and env.resources["diamond"] >= 5:
            print("Goal achieved: Gathered 5 diamonds")
        else:
            print("Goal not achieved. The agent may need to generate a new plan or adjust its strategy.")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Starting Voyager AGI simulation...

Current goal: Craft a stone pickaxe
Current state:
Resources: {'wood': 0, 'stone': 0, 'iron': 0, 'diamond': 0, 'coal': 0}
Crafted items: {'wooden_pickaxe': 0, 'stone_pickaxe': 0, 'iron_pickaxe': 0, 'furnace': 0}

Generated plan:
- gather wood (amount: 1)
- gather stone (amount: 3)
- craft wooden_pickaxe
- gather coal (amount: 2)
- craft furnace
- gather iron (amount: 3)
- craft iron_pickaxe
- gather stone (amount: 3)
- craft stone_pickaxe

Executing plan:
- Gathered 1 wood
- Gathered 3 stone
- Cannot craft wooden_pickaxe
- Gathered 2 coal
- Cannot craft furnace
- Gathered 3 iron
- Cannot craft iron_pickaxe
- Gathered 3 stone
- Cannot craft stone_pickaxe

Reflection:
Resources: {'wood': 1, 'stone': 6, 'iron': 3, 'diamond': 0, 'coal': 2}
Crafted items: {'wooden_pickaxe': 0, 'stone_pickaxe': 0, 'iron_pickaxe': 0, 'furnace': 0}
Goal not achieved. The agent may need to generate a new plan or adjust its strategy.

Current goal: Gather 10 iron ore
Current s